# Análise Exploratória de Dados

## Preparação

In [1]:
import polars as pl

from df_utils import get_list_column_max_len, one_hot_encode_list_column
from geo_location import extract_polygons_from_folder, check_point_in_kml_polygon

## Leitura de dados

Os dados se encontram no formato NDJSON, também conhecido como JSONLines,
em que há um objeto JSON por linha no arquivo.

In [2]:
neighborhoods = ["cidade_baixa", "centro_historico", "menino_deus", "sarandi"]

neighborhoods_df_map = {
    neighborhood: pl.read_ndjson(f"data/{neighborhood}_listings.json")
    for neighborhood in neighborhoods
}

In [3]:
try:
    pl.concat(neighborhoods_df_map.values())
except pl.exceptions.SchemaError:
    print("Dados não têm esquema homogêneo.")

Dados não têm esquema homogêneo.


In [4]:
for name, df in neighborhoods_df_map.items():
    print(name)
    print("=" * len(name))
    print(df.schema)
    print()

cidade_baixa
Schema([('listing', Struct({'parkingSpaces': List(Int64), 'floors': List(Int64), 'unitsOnTheFloor': Int64, 'unitTypes': List(String), 'advertiserContact': Struct({'chat': String, 'phones': List(String)}), 'stamps': List(String), 'displayAddressType': String, 'capacityLimit': List(Int64), 'providerId': String, 'buildings': Int64, 'unitSubTypes': List(String), 'contractType': String, 'publicationType': String, 'totalAreas': List(Int64), 'createdAt': String, 'amenities': List(String), 'description': String, 'whatsappNumber': String, 'title': String, 'usableAreas': List(Int64), 'legacyId': String, 'updatedAt': String, 'address': Struct({'city': String, 'neighborhood': String, 'street': String, 'streetNumber': String, 'point': Struct({'source': String, 'approximateLat': Float64, 'approximateLon': Float64, 'radius': Int64, 'lat': Float64, 'lon': Float64}), 'stateAcronym': String}), 'advertiserId': String, 'status': String, 'constructionStatus': String, 'resale': Boolean, 'suites

A maior diferença parece estar na estrutura aninhada `rentalInfo`,
que não é presente nos dados do bairro Sarandi.

## Entendimento dos dados e pré-processamento

In [5]:
neighborhoods_df_map["cidade_baixa"].head(2)

listing,account,medias,accountLink,link
struct[43],struct[9],list[struct[3]],struct[4],struct[4]
"{[],[8],4,[""APARTMENT""],{"""",[""5130130100""]},[],""STREET"",[0],"""",0,[],""OWNER"",""PREMIUM"",[48],""2019-11-05T13:09:45.557+00:00"",[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""],""Apartamento em excelente estado, iluminado e silencioso, mobiliado com móveis embutidos , conceito aberto, cozinha e sala integrados, gás central, aquecimento em todas as torneiras, fica ar condicionado do quarto e todos os móveis embutidos quarto, banheiro, sala, cozinha e homeoffice."","""",""Apartamento de 46 metros quadrados no bairro Cidade Baixa com 1 quarto"",[46],""26349587"",""2024-12-17T11:12:11.169+00:00"",{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""},""dfadbe19-2554-5480-56eb-0780d0ee4019"",""ACTIVE"",""ConstructionStatus_NONE"",false,[],""ZAP1593841"",[1],""2463567709"",true,[1],0,[{""SALE"",600,290000,300,null}],""USED"",null,""NonActivationReason_NONE"",""GRUPOZAP"",[""RESIDENTIAL""],true,""a5adbade-589d-3c61-97b7-e4014f21cf3d"",""UNIT""}","{""dfadbe19-2554-5480-56eb-0780d0ee4019"",""BÁRBARA EINSFELD DE BORBA"",null,"""",true,345436,3136761,""2018-05-11T05:14:21Z"",""""}","[{""7cb4b1098bf75603a7f2561456bc2883"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/7cb4b1098bf75603a7f2561456bc2883.webp"",""IMAGE""}, {""3ad45448360d515b4bbb3d833537b5e4"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3ad45448360d515b4bbb3d833537b5e4.webp"",""IMAGE""}, … {""3c74183638571539a2c2f9f568a72d7a"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3c74183638571539a2c2f9f568a72d7a.webp"",""IMAGE""}]","{""BÁRBARA EINSFELD DE BORBA"",""/imobiliaria/345436/"","""",{}}","{""Apartamento com 1 Quarto à venda, 46m²"",""/imovel/venda-apartamento-1-quarto-mobiliado-cidade-baixa-porto-alegre-46m2-id-2463567709/"","""",{""Porto Alegre"",""Cidade Baixa"","""",""Rua Joaquim Nabuco"","""",""""}}"
"{[],[],0,[""APARTMENT""],{"""",[""5132084035"", ""51999671489""]},[],""ALL"",[],""13887"",0,[],""REAL_ESTATE"",""PREMIUM"",[66],""2024-12-01T21:22:02.001+00:00"",[],""Compre apartamento silencioso,55,10m2 de área útil,2 quartos, piso parquet,living 2 ambientes,banheiro social ,cozinha e área de serviço. O condomínio fica localizado em Rua José do Patrocínio no bairro Cidade Baixa em Porto Alegre. Está bem situado, próximo a pontos de interesse de Cidade Baixa, tais como Azambuja, Praça General Daltro Filho, Faculdade de Direito da Funda. Escola Sup. do Ministério Público, Praça Salvador Allende, e Policlínica Militar de Porto Alegre.Aceitamos Fiinanciamento Bancário e Fgts.Marque hoje mesmo uma visita com um de nossos Consultores Imobiliários caddstrados! ]]>"",""51999671489"",""Compre apartamento silencioso,55,10m2 de área útil,2 quartos, piso parquet,living 2 ambientes,banhei"",[56],"""",""2024-12-26T12:00:41.422+00:00"",{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""},""90879777-b9cc-65b9-28c8-c86f00947af9"",""ACTIVE"",""ConstructionStatus_NONE"",false,[0],""VR527903"",[1],""2760694515"",true,[2],0,[{""SALE"",0,275000,290,null}],""USED"",7,""NonActivationReason_NONE"",""GRUPOZAP"",[""RESIDENTIAL""],false,""40017671-36da-3cb2-98e7-116dd4abcba1"",""UNIT""}","{""90879777-b9cc-65b9-28c8-c86f00947af9"",""Sperinde Vendas"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/fb5adab24465e4ba59ba7d6e4ac1de9e.webp"",""00411-J-RS"",true,33518,2594637,""2018-03-27T18:49:45Z"",""diamond""}","[{""3d8f346437754cf315cc942816497178"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3d8f346437754cf315cc942816497178.webp"",""IMAGE""}, {""d309f06d26092646e716b47e93936c7d"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/d309f06d26092646e716b4

### Qual o tamanho da massa de dados?

In [6]:
total = 0
for neighborhood, df in neighborhoods_df_map.items():
    df_height = df.select(pl.len()).item()
    print(f"{neighborhood}: {df_height}")
    total += df_height
print(f"Total: {total}")

cidade_baixa: 2790
centro_historico: 6007
menino_deus: 5833
sarandi: 2711
Total: 17341


A massa de dados é relativamente pequena, totalizando 17.341 imóveis.

In [7]:
columns = neighborhoods_df_map["cidade_baixa"].schema.names()
print(f'Temos {len(columns)} colunas: {", ".join(columns)}.')

Temos 5 colunas: listing, account, medias, accountLink, link.


### Pré-processamento inicial

Quase todas as informações que consideramos importantes para nossa análise estão na coluna `listing`,
que tem uma estrutura aninhada complexa. Aplainamos a estrutura para simplificar a análise,
e mantemos o link do anúncio para facilitar a consulta para análises mais aprofundadas.

In [8]:
neighborhoods_df_map["centro_historico"].select(pl.col("link").struct.unnest()).head(4)

name,href,rel,data
str,str,str,struct[6]
"""Conjunto Comercial / Sala à ve…","""/imovel/venda-conjunto-comerci…","""""","{""Porto Alegre"",""Centro Histórico"","""",""Praça Quinze de Novembro"",""16"",""""}"
"""Conjunto Comercial / Sala à ve…","""/imovel/venda-conjunto-comerci…","""""","{""Porto Alegre"",""Centro Histórico"","""",""Rua dos Andradas"",""1234"",""""}"
"""Conjunto Comercial / Sala à ve…","""/imovel/venda-conjunto-comerci…","""""","{""Porto Alegre"",""Centro Histórico"","""",""Rua Marechal Floriano Peixoto"",""13"",""""}"
"""Prédio Inteiro à venda, 2026m²""","""/imovel/venda-predio-com-eleva…","""""","{""Porto Alegre"",""Centro Histórico"","""",""Avenida Júlio de Castilhos"",""48"",""""}"


In [9]:
listings_df_map = {
    neighborhood: df.select([
        pl.col("listing").struct.unnest(),
        pl.col("link").struct.field("href").alias("link_href")
    ])
    for neighborhood, df in neighborhoods_df_map.items()
}

In [10]:
for name, df in listings_df_map.items():
    listings_df_map[name] = listings_df_map[name].with_columns(
        pl.lit(name).alias("neighborhood")
    )

In [11]:
listings_df_map["centro_historico"].head(3)

parkingSpaces,floors,unitsOnTheFloor,unitTypes,advertiserContact,stamps,displayAddressType,capacityLimit,providerId,buildings,unitSubTypes,contractType,publicationType,totalAreas,createdAt,amenities,description,whatsappNumber,title,usableAreas,legacyId,updatedAt,address,advertiserId,status,constructionStatus,resale,suites,externalId,bathrooms,id,showPrice,bedrooms,unitFloor,pricingInfos,listingType,listingsCount,nonActivationReason,portal,usageTypes,acceptExchange,sourceId,propertyType,link_href,neighborhood
list[i64],list[i64],i64,list[str],struct[2],list[str],str,list[null],str,i64,list[str],str,str,list[i64],str,list[str],str,str,str,list[i64],str,str,struct[6],str,str,str,bool,list[i64],str,list[i64],str,bool,list[i64],i64,list[struct[5]],str,i64,str,str,list[str],bool,str,str,str,str
[],[],0,"[""OFFICE""]","{"""",[""51982460700""]}",[],"""ALL""",[],"""76119""",0,[],"""REAL_ESTATE""","""PREMIUM""",[66],"""2024-12-10T23:00:53.946+00:00""","[""INTERCOM"", ""ELEVATOR"", ""CARETAKER""]","""Explore o potencial de investi…","""51982460700""","""Sala Comercial - Centro Histór…",[49],"""""","""2024-12-17T23:18:49.917+00:00""","{""Porto Alegre"",""Centro Histórico"",""Praça Quinze de Novembro"",""16"",{""GOOGLE"",null,null,null,-30.027907,-51.226606},""RS""}","""d178a00c-2318-8959-c584-449a9f…","""ACTIVE""","""ConstructionStatus_NONE""",false,[],"""74060538""",[],"""2763007925""",true,[],0,"[{""SALE"",1575,131790,0,null}]","""USED""",null,"""NonActivationReason_NONE""","""GRUPOZAP""","[""COMMERCIAL""]",false,"""f3ff3a42-8142-33f0-9d16-74d954…","""UNIT""","""/imovel/venda-conjunto-comerci…","""centro_historico"""
[],[],0,"[""OFFICE""]","{"""",[""5131913045"", ""51995540021""]}",[],"""ALL""",[],"""43018""",0,[],"""REAL_ESTATE""","""STANDARD""",[1083],"""2024-10-31T19:46:30.211+00:00""",[],"""Crédito Real vende sala comerc…","""51995540021""","""Sala comercial com 820 m² priv…",[821],"""""","""2024-12-26T10:39:14.438+00:00""","{""Porto Alegre"",""Centro Histórico"",""Rua dos Andradas"",""1234"",{""GOOGLE"",null,null,null,-30.029844,-51.229421},""RS""}","""c10bd73f-1704-1915-de4b-5f4085…","""ACTIVE""","""ConstructionStatus_NONE""",false,[],"""10003930""",[],"""2753265605""",true,[],0,"[{""SALE"",3235,2900000,0,null}]","""USED""",null,"""NonActivationReason_NONE""","""GRUPOZAP""","[""COMMERCIAL""]",false,"""21b31137-f150-363e-b477-02473d…","""UNIT""","""/imovel/venda-conjunto-comerci…","""centro_historico"""
[],[],0,"[""OFFICE""]","{"""",[""51997974224""]}",[],"""ALL""",[],"""56822""",0,[],"""REAL_ESTATE""","""STANDARD""",[167],"""2024-05-17T02:47:07.305+00:00""","[""KITCHEN""]","""Sala / Conjunto Comercial a ve…","""51997974224""","""SALA / CONJUNTO COMERCIAL NO M…",[61],"""""","""2024-12-26T01:38:37.135+00:00""","{""Porto Alegre"",""Centro Histórico"",""Rua Marechal Floriano Peixoto"",""13"",{""GOOGLE"",null,null,null,-30.028941,-51.22681},""RS""}","""abd88457-13be-a5c6-a70f-f484ce…","""ACTIVE""","""ConstructionStatus_NONE""",false,[],"""9937782""",[2],"""2715472864""",true,[],0,"[{""SALE"",1400,179000,430,null}]","""USED""",null,"""NonActivationReason_NONE""","""GRUPOZAP""","[""COMMERCIAL""]",false,"""27d64c87-9b95-3d35-af44-cc072c…","""UNIT""","""/imovel/venda-conjunto-comerci…","""centro_historico"""


#### Descarte de informações não importantes

Existe muita informação ruidosa nesses dados.
Escolhemos as seguintes colunas para descarte.

In [12]:
COLUMNS_TO_DROP = [
    "advertiserId",
    "constructionStatus",
    "portal",
    "stamps",
    "advertiserContact",
    "whatsappNumber",
    "title",
    "nonActivationReason",
    "status",
    "legacyId",
    "externalId",
    "listingsCount",
    "createdAt",
    "updatedAt",
    "showPrice",
    "acceptExchange",
    "description",
    "sourceId",
    "providerId"
]

listings_df_map = {
    name: df.drop(COLUMNS_TO_DROP)
    for name, df in listings_df_map.items()
}

#### Atributos em forma de lista

Sabemos que `amenities` descrevem as amenidades presentes em um dado imóvel,
o que pode ser interessante transformar em atributos preditores para nossas regressões.

Entretanto, ainda precisamos entender outras colunas que tem listas como valores.
Várias delas não parecem fazer sentido em ter multiplicidade: em que contexto o número
de quartos (bedrooms) precisa ser uma lista? Número de banheiros? Área utilizável?
Uma unidade pode pertencer ao mesmo tempo a mais de um tipo (comercial, casa)?

Em primeiro lugar, tentamos entender quais listas de fato vêm a possuir mais de um
elemento.


In [13]:
LIST_COLUMNS = [
    "floors",
    "unitSubTypes",
    "suites",
    "unitTypes",
    "pricingInfos",
    "parkingSpaces",
    "totalAreas",
    "bathrooms",
    "bedrooms",
    "usableAreas",
    "usageTypes",
    "capacityLimit",
]

max_colname_width = max(len(name) for name in LIST_COLUMNS)
arrays_with_size_larger_than_one = []
for name, df in listings_df_map.items():
    print(name)
    print("=" * len(name))
    for colname in LIST_COLUMNS:
        column_max_length = get_list_column_max_len(df, colname)
        print(f"{colname:<{max_colname_width}}: {column_max_length}")
        if column_max_length > 1:
            arrays_with_size_larger_than_one.append(colname)
    print("\n")

cidade_baixa
floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1


centro_historico
floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1


menino_deus
floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1


sarandi
floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1




A primeira coisa a se fazer, para facilitar o restante da nossa análise e pré-processamento,
é transformar aquelas colunas que são sempre listas de um elemento em colunas escalares.

In [14]:
single_element_list_columns = [
    "capacityLimit",
    "unitTypes",
    "floors",
    "suites",
]

for name, df in listings_df_map.items():
    listings_df_map[name] = df.with_columns(
        [pl.col(col).list.first().alias(col) for col in single_element_list_columns]
    )

In [15]:
listings_df_map["cidade_baixa"].head(5)

parkingSpaces,floors,unitsOnTheFloor,unitTypes,displayAddressType,capacityLimit,buildings,unitSubTypes,contractType,publicationType,totalAreas,amenities,usableAreas,address,resale,suites,bathrooms,id,bedrooms,unitFloor,pricingInfos,listingType,usageTypes,propertyType,link_href,neighborhood
list[i64],i64,i64,str,str,i64,i64,list[str],str,str,list[i64],list[str],list[i64],struct[6],bool,i64,list[i64],str,list[i64],i64,list[struct[5]],str,list[str],str,str,str
[],8,4,"""APARTMENT""","""STREET""",0,0,[],"""OWNER""","""PREMIUM""",[48],"[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""]",[46],"{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""}",false,null,[1],"""2463567709""",[1],0,"[{""SALE"",600,290000,300,null}]","""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-1-qu…","""cidade_baixa"""
[],null,0,"""APARTMENT""","""ALL""",null,0,[],"""REAL_ESTATE""","""PREMIUM""",[66],[],[56],"{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""}",false,0,[1],"""2760694515""",[2],0,"[{""SALE"",0,275000,290,null}]","""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-2-qu…","""cidade_baixa"""
[],null,0,"""APARTMENT""","""ALL""",null,0,[],"""REAL_ESTATE""","""STANDARD""",[56],"[""PETS_ALLOWED"", ""FURNISHED""]",[42],"{""Porto Alegre"",""Centro Histórico"",""Avenida João Pessoa"",""371"",{""GOOGLE"",null,null,null,-30.035694,-51.221204},""RS""}",false,null,[1],"""2765177708""",[1],3,"[{""SALE"",335,175000,530,null}]","""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-1-qu…","""cidade_baixa"""
[1],null,0,"""APARTMENT""","""ALL""",null,0,[],"""REAL_ESTATE""","""STANDARD""",[61],"[""GARAGE""]",[46],"{""Porto Alegre"",""Cidade Baixa"",""Rua Miguel Teixeira"",""66"",{""GOOGLE"",null,null,null,-30.042558,-51.227121},""RS""}",false,1,[2],"""2756577162""",[1],0,"[{""SALE"",113,380000,400,null}]","""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-1-qu…","""cidade_baixa"""
[],null,0,"""OFFICE""","""ALL""",null,0,"[""OFFICE""]","""OWNER""","""PREMIUM""",[],[],[130],"{""Porto Alegre"",""Cidade Baixa"",""Travessa do Carmo"",""152"",{""GOOGLE"",null,null,null,-30.038998,-51.226354},""RS""}",true,null,[2],"""2708598936""",[],4,"[{""SALE"",null,500000,null,null}]","""USED""","[""COMMERCIAL""]","""UNIT""","""/imovel/venda-conjunto-comerci…","""cidade_baixa"""


A seguir, procuramos entender a coluna `pricingInfos`.

In [16]:
listings_df_map["centro_historico"].filter(
    pl.col("pricingInfos").list.len() > 1
).head(3)

parkingSpaces,floors,unitsOnTheFloor,unitTypes,displayAddressType,capacityLimit,buildings,unitSubTypes,contractType,publicationType,totalAreas,amenities,usableAreas,address,resale,suites,bathrooms,id,bedrooms,unitFloor,pricingInfos,listingType,usageTypes,propertyType,link_href,neighborhood
list[i64],i64,i64,str,str,null,i64,list[str],str,str,list[i64],list[str],list[i64],struct[6],bool,i64,list[i64],str,list[i64],i64,list[struct[5]],str,list[str],str,str,str
[0],null,0,"""OFFICE""","""ALL""",null,0,[],"""REAL_ESTATE""","""PREMIUM""",[42],[],[42],"{""Porto Alegre"",""Centro Histórico"",""Rua dos Andradas"",""1137"",{""GOOGLE"",null,null,null,-30.030361,-51.230258},""RS""}",false,0,[1],"""2712201134""",[0],0,"[{""RENTAL"",1289,1200,500,{""MONTHLY"",[],1700}}, {""SALE"",1289,160000,500,null}]","""USED""","[""COMMERCIAL""]","""UNIT""","""/imovel/venda-conjunto-comerci…","""centro_historico"""
[2],null,0,"""OFFICE""","""ALL""",null,0,[],"""REAL_ESTATE""","""STANDARD""",[35],[],[35],"{""Porto Alegre"",""Cidade Baixa"",""Avenida Loureiro da Silva"",""2001"",{""GOOGLE"",null,null,null,-30.035203,-51.221879},""RS""}",false,0,[1],"""2570524388""",[0],0,"[{""RENTAL"",331,1350,739,{""MONTHLY"",[],2089}}, {""SALE"",331,265000,739,null}]","""USED""","[""COMMERCIAL""]","""UNIT""","""/imovel/venda-conjunto-comerci…","""centro_historico"""
[1],22,0,"""OFFICE""","""STREET""",null,0,[],"""REAL_ESTATE""","""STANDARD""",[65],"[""ELEVATOR""]",[52],"{""Porto Alegre"",""Centro Histórico"",""Avenida Senador Salgado Filho"",null,{""GOOGLE"",-30.031,-51.224,140,null,null},""RS""}",false,0,[2],"""2743763450""",[0],0,"[{""RENTAL"",150,1100,160,{""MONTHLY"",[],1260}}, {""SALE"",150,275000,160,null}]","""USED""","[""COMMERCIAL""]","""UNIT""","""/imovel/venda-conjunto-comerci…","""centro_historico"""


In [17]:
listings_df_map["centro_historico"].select(
    pl.col("pricingInfos").explode().struct.field("businessType").unique()
)

businessType
str
"""SALE"""
"""RENTAL"""


Quando ocorre multiplicidade, é porque o imóvel pode ser comprado ou alugado.
Para nossos propósitos, nos interessamos tão somente em propriedades à venda, e apenas no valor de compra.
Desta forma, podemos descartar todas as outras informações e imóveis que não estão disponíveis para compra.

In [18]:
for name, df in listings_df_map.items():
    listings_df_map[name] = (
        df.explode(pl.col("pricingInfos"))
        .unnest("pricingInfos")
        .filter(pl.col("businessType") != "RENTAL")
    )

Lembrando que os dados dos bairros diferem em `rentalInfo`, ausente no bairro Sarandi.
Podemos descartar esta informação, mantendo apenas o preço do imóvel.

In [19]:
for name,df in listings_df_map.items():
    if name != "sarandi":
        listings_df_map[name] = listings_df_map[name].drop("rentalInfo")

E como os esquemas dos dados diferiam apenas em `rentalInfo`,
agora podemos concatenar os dados em um único DataFrame para facilitar nossa análise.

In [20]:
df = pl.concat(listings_df_map.values())

In [22]:
df.head(3)

parkingSpaces,floors,unitsOnTheFloor,unitTypes,displayAddressType,capacityLimit,buildings,unitSubTypes,contractType,publicationType,totalAreas,amenities,usableAreas,address,resale,suites,bathrooms,id,bedrooms,unitFloor,businessType,yearlyIptu,price,monthlyCondoFee,listingType,usageTypes,propertyType,link_href,neighborhood
list[i64],i64,i64,str,str,i64,i64,list[str],str,str,list[i64],list[str],list[i64],struct[6],bool,i64,list[i64],str,list[i64],i64,str,i64,i64,i64,str,list[str],str,str,str
[],8,4,"""APARTMENT""","""STREET""",0,0,[],"""OWNER""","""PREMIUM""",[48],"[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""]",[46],"{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""}",false,null,[1],"""2463567709""",[1],0,"""SALE""",600,290000,300,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-1-qu…","""cidade_baixa"""
[],null,0,"""APARTMENT""","""ALL""",null,0,[],"""REAL_ESTATE""","""PREMIUM""",[66],[],[56],"{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""}",false,0,[1],"""2760694515""",[2],0,"""SALE""",0,275000,290,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-2-qu…","""cidade_baixa"""
[],null,0,"""APARTMENT""","""ALL""",null,0,[],"""REAL_ESTATE""","""STANDARD""",[56],"[""PETS_ALLOWED"", ""FURNISHED""]",[42],"{""Porto Alegre"",""Centro Histórico"",""Avenida João Pessoa"",""371"",{""GOOGLE"",null,null,null,-30.035694,-51.221204},""RS""}",false,null,[1],"""2765177708""",[1],3,"""SALE""",335,175000,530,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-1-qu…","""cidade_baixa"""


In [23]:
df = df.drop("businessType", "monthlyCondoFee", "yearlyIptu")

A seguir, buscamos entender `unitSubTypes`.

In [24]:
df.filter(pl.col("unitSubTypes").list.len() > 1).head(3)

parkingSpaces,floors,unitsOnTheFloor,unitTypes,displayAddressType,capacityLimit,buildings,unitSubTypes,contractType,publicationType,totalAreas,amenities,usableAreas,address,resale,suites,bathrooms,id,bedrooms,unitFloor,price,listingType,usageTypes,propertyType,link_href,neighborhood
list[i64],i64,i64,str,str,i64,i64,list[str],str,str,list[i64],list[str],list[i64],struct[6],bool,i64,list[i64],str,list[i64],i64,i64,str,list[str],str,str,str
[1],4,1,"""APARTMENT""","""ALL""",null,1,"[""PENTHOUSE"", ""DUPLEX""]","""REAL_ESTATE""","""PREMIUM""","[114, 115]","[""GARDEN"", ""NEAR_SCHOOL"", … ""BATHROOM_CABINETS""]","[100, 101]","{""Porto Alegre"",""Centro Histórico"",""Rua Sarmento Leite"",""951"",{""GOOGLE"",null,null,null,-30.037074,-51.223901},""RS""}",false,0,[1],"""2636785204""",[1],3,429000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-cobertura-1-quar…","""cidade_baixa"""
[1],9,4,"""APARTMENT""","""STREET""",null,0,"[""PENTHOUSE"", ""DUPLEX""]","""REAL_ESTATE""","""STANDARD""",[252],"[""SECURITY_CAMERA"", ""BARBECUE_GRILL"", … ""FIREPLACE""]",[192],"{""Porto Alegre"",""Centro Histórico"",""Rua Avaí"",null,{""GOOGLE"",-30.034,-51.222,140,null,null},""RS""}",false,1,[3],"""2651401754""",[3],7,990000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-cobertura-3-quar…","""cidade_baixa"""
[1],6,8,"""APARTMENT""","""ALL""",null,1,"[""LOFT"", ""DUPLEX""]","""REAL_ESTATE""","""STANDARD""",[57],"[""SECURITY_CAMERA"", ""SAFETY_CIRCUIT"", … ""AMERICAN_KITCHEN""]",[57],"{""Porto Alegre"",""Azenha"",""Avenida Venâncio Aires"",""29"",{""GOOGLE"",null,null,null,-30.043109,-51.219469},""RS""}",false,1,[2],"""2751040007""",[1],6,580000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-loft-1-quarto-co…","""cidade_baixa"""


`unitSubTypes`, neste caso, define duas categorias que se aplicam ao imóvel.

Por outro lado, um imóvel pode não ter nenhum subtipo? E, se sim, isso é comum?

In [25]:
df.filter(pl.col("unitSubTypes").list.len() == 0).height

16561

De fato, esse é o mais comum dos casos. Se esse dado fosse presente para todos os registros,
poderia ser um atributo interessante. Como não é o caso, o descartamos.

In [26]:
df = df.drop("unitSubTypes")

In [27]:
df.head(3)

parkingSpaces,floors,unitsOnTheFloor,unitTypes,displayAddressType,capacityLimit,buildings,contractType,publicationType,totalAreas,amenities,usableAreas,address,resale,suites,bathrooms,id,bedrooms,unitFloor,price,listingType,usageTypes,propertyType,link_href,neighborhood
list[i64],i64,i64,str,str,i64,i64,str,str,list[i64],list[str],list[i64],struct[6],bool,i64,list[i64],str,list[i64],i64,i64,str,list[str],str,str,str
[],8,4,"""APARTMENT""","""STREET""",0,0,"""OWNER""","""PREMIUM""",[48],"[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""]",[46],"{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""}",false,null,[1],"""2463567709""",[1],0,290000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-1-qu…","""cidade_baixa"""
[],null,0,"""APARTMENT""","""ALL""",null,0,"""REAL_ESTATE""","""PREMIUM""",[66],[],[56],"{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""}",false,0,[1],"""2760694515""",[2],0,275000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-2-qu…","""cidade_baixa"""
[],null,0,"""APARTMENT""","""ALL""",null,0,"""REAL_ESTATE""","""STANDARD""",[56],"[""PETS_ALLOWED"", ""FURNISHED""]",[42],"{""Porto Alegre"",""Centro Histórico"",""Avenida João Pessoa"",""371"",{""GOOGLE"",null,null,null,-30.035694,-51.221204},""RS""}",false,null,[1],"""2765177708""",[1],3,175000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-1-qu…","""cidade_baixa"""


In [28]:
df.filter(pl.col("bedrooms").list.len() > 1).head(5)

parkingSpaces,floors,unitsOnTheFloor,unitTypes,displayAddressType,capacityLimit,buildings,contractType,publicationType,totalAreas,amenities,usableAreas,address,resale,suites,bathrooms,id,bedrooms,unitFloor,price,listingType,usageTypes,propertyType,link_href,neighborhood
list[i64],i64,i64,str,str,i64,i64,str,str,list[i64],list[str],list[i64],struct[6],bool,i64,list[i64],str,list[i64],i64,i64,str,list[str],str,str,str
[1],10,4,"""APARTMENT""","""ALL""",null,1,"""REAL_ESTATE""","""PREMIUM""","[98, 128]","[""GARDEN"", ""BEDROOM_WARDROBE"", … ""CABLE_TV""]","[97, 99]","{""Porto Alegre"",""Cidade Baixa"",""Rua Luiz Afonso"",""158"",{""GOOGLE"",null,null,null,-30.038824,-51.220484},""RS""}",false,1,"[1, 3]","""2711554807""","[2, 3]",7,530000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-3-qu…","""cidade_baixa"""
[2],null,0,"""OFFICE""","""ALL""",null,0,"""REAL_ESTATE""","""STANDARD""",[95],"[""GARAGE"", ""AIR_CONDITIONING"", … ""INTERNET_ACCESS""]",[37],"{""Porto Alegre"",""Cidade Baixa"",""Avenida Loureiro da Silva"",""2001"",{""GOOGLE"",null,null,null,-30.035203,-51.221879},""RS""}",false,null,[1],"""2670004814""","[0, 2]",8,220000,"""USED""","[""COMMERCIAL""]","""UNIT""","""/imovel/venda-conjunto-comerci…","""cidade_baixa"""
"[0, 1]",null,0,"""APARTMENT""","""ALL""",null,0,"""REAL_ESTATE""","""STANDARD""","[67, 125]","[""GARAGE""]","[73, 106]","{""Porto Alegre"",""Azenha"",""Rua Lobo da Costa"",""279"",{""GOOGLE"",null,null,null,-30.04323,-51.215839},""RS""}",false,1,"[1, 3]","""2734713180""","[2, 3]",0,469000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-apartamento-3-qu…","""cidade_baixa"""
[2],null,0,"""HOME""","""ALL""",null,0,"""REAL_ESTATE""","""PREMIUM""","[440, 610]","[""GARAGE""]","[439, 441]","{""Porto Alegre"",""Azenha"",""Rua Doutor Sebastião Leão"",""233"",{""GOOGLE"",null,null,null,-30.044312,-51.215602},""RS""}",false,2,"[4, 5]","""2762298733""","[3, 4]",0,990000,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-casa-4-quartos-a…","""cidade_baixa"""
[0],9,0,"""APARTMENT""","""ALL""",null,0,"""REAL_ESTATE""","""STANDARD""",[43],"[""PARTY_HALL"", ""INTERCOM""]",[31],"{""Porto Alegre"",""Cidade Baixa"",""Rua João Alfredo"",""312"",{""GOOGLE"",null,null,null,-30.039851,-51.226294},""RS""}",false,0,[1],"""2745508198""","[0, 1]",0,162450,"""USED""","[""RESIDENTIAL""]","""UNIT""","""/imovel/venda-quitinete-1-quar…","""cidade_baixa"""


Existem dados duplicados?

In [16]:
for name, df in listings_df_map.items():
    num_dupes = df.filter(pl.col("id").is_duplicated()).height
    print(f"{name}: {num_dupes} duplicatas")

cidade_baixa: 102 duplicatas
centro_historico: 712 duplicatas
menino_deus: 904 duplicatas
sarandi: 132 duplicatas


In [37]:
neighborhoods_df_map["cidade_baixa"].filter(pl.col("listing").struct.field("id") == "2761415335")

listing,account,medias,accountLink,link
struct[43],struct[9],list[struct[3]],struct[4],struct[4]
"{""10929a8c-7227-5aff-fb05-6bf3945bf5b7"",[1],[2],""UNIT"",false,[104],"""",[],""83977"",""2024-12-04T20:20:07.087+00:00"",""ConstructionStatus_NONE"",[],""NonActivationReason_NONE"",{""Porto Alegre"",""Farroupilha"",""Avenida Venâncio Aires"",""449"",{""GOOGLE"",null,null,null,-30.041217,-51.215602},""RS""},[0],0,""51992941900"",false,""ACTIVE"",""2024-12-25T20:16:09.176+00:00"",[],[82],""ALL"",7,""USED"",""Excelente oportunidade no coração de Porto Alegre!<BR><BR>Este apartamento encantador de 81m² de área privativa está localizado na Avenida Venâncio Aires, em uma região privilegiada, próximo à Avenida João Pessoa, Parque Farroupilha (Redenção) e ao bairro Bom Fim.<BR><BR>Destaques do imóvel:<BR>02 dormitórios espaçosos;<BR>Amplo living com 02 ambientes (estar e jantar);<BR>Cozinha separada e área de serviço ampliada;<BR>Piso em parquet, que adiciona charme e conforto aos ambientes;<BR>Imóvel semimobiliado, bem ventilado, ensolarado e reformado.<BR><BR>Localização estratégica:<BR>Fácil acesso aos principais serviços, comércio e lazer dos bairros Cidade Baixa, Santana e Bom Fim.<BR><BR>Ideal para quem busca conforto, praticidade e qualidade de vida!<BR><BR>Entre em contato e agende sua visita. Não perca essa oportunidade de viver bem em Porto Alegre! ]]>"",[],""REAL_ESTATE"",[""RESIDENTIAL""],0,[],""STANDARD"",""Excelente oportunidade no coração de Porto Alegre!Este apartamento encantador de 81m² de área privat"",""2761415335"",""c6773be8-271d-36e2-95b9-07d1d405165c"",0,[],{"""",[""5137373337"", ""51992941900""]},[{""SALE"",0,430000,550,null}],true,""GRUPOZAP"",""BE11051"",[""APARTMENT""]}","{""10929a8c-7227-5aff-fb05-6bf3945bf5b7"",""Orientta Negócios Imobiliários"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/700237e1377fef8be63496af3d015bcc.webp"",""36071-F-RS"",true,54505,2594899,""2018-03-27T16:55:54Z"",""gold""}","[{""02cb37650ba179846553f62e62610c71"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/02cb37650ba179846553f62e62610c71.webp"",""IMAGE""}, {""a6593237d8c622c272b7149661d39ffc"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/a6593237d8c622c272b7149661d39ffc.webp"",""IMAGE""}, … {""ae5c0129d8206466ddaa81ed822f9c90"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/ae5c0129d8206466ddaa81ed822f9c90.webp"",""IMAGE""}]","{""Orientta Negócios Imobiliários"",""/imobiliaria/54505/"","""",{}}","{""Apartamento com 2 Quartos à venda, 82m²"",""/imovel/venda-apartamento-2-quartos-farroupilha-porto-alegre-82m2-id-2761415335/"","""",{""Porto Alegre"",""Farroupilha"","""",""Avenida Venâncio Aires"",""449"",""""}}"
"{""10929a8c-7227-5aff-fb05-6bf3945bf5b7"",[1],[2],""UNIT"",false,[102],"""",[],""83977"",""2024-12-04T20:20:07.087+00:00"",""ConstructionStatus_NONE"",[],""NonActivationReason_NONE"",{""Porto Alegre"",""Farroupilha"",""Avenida Venâncio Aires"",""449"",{""GOOGLE"",null,null,null,-30.041217,-51.215602},""RS""},[0],0,""51992941900"",false,""ACTIVE"",""2024-12-26T20:27:47.915+00:00"",[],[80],""ALL"",2,""USED"",""Excelente oportunidade no coração de Porto Alegre!<BR><BR>Este apartamento encantador de 81m² de área privativa está localizado na Avenida Venâncio Aires, em uma região privilegiada, próximo à Avenida João Pessoa, Parque Farroupilha (Redenção) e ao bairro Bom Fim.<BR><BR>Destaques do imóvel:<BR>02 dormitórios espaçosos;<BR>Amplo living com 02 ambientes (estar e jantar);<BR>Cozinha separada e área de serviço ampliada;<BR>Piso em parquet, que adiciona charme e conforto aos ambientes;<BR>Imóvel semimobiliado, bem ventilado, ensolarado e reformado.<BR><BR>Localização estratégica:<BR>Fácil acesso aos principais serviços, comércio e lazer dos bairros Cidade Baixa, Santana e Bom Fim.<BR><BR>Ideal para quem busca conforto, praticidade e qualidade de vida!<BR><BR>Entre em contato e agende sua visita

In [ ]:
kml_data_file = "data/cheias_em_porto_alegre.kml"
polygons = extract_polygons_from_folder(
    kml_data_file, "Inundação simulada com nível 500 cm (5.0 m)"
)